This notebook is for testing the application. You can play around with it. 

Make sure to have your api key and your model name configured in `services/config/personal_constants.py`

In [2]:
# importing the class from a different file
from services.querier.querier import DbQuerier

# instantiating the class
test_querier = DbQuerier(sample_db_loc='data/PostNL_SQLite.sqlite')

# do some configurations
test_querier.remove_all_training_data() # if there was any, now there is not
test_querier.train_model(train_on_documentation=True, 
                         train_on_ddl=True, 
                         train_on_sql=True, 
                         train_on_question_sql_pairs=False)

In [8]:
# showing the training data that is in the model
test_querier.training_data

,id,training_data_type,question,content
0,203102-sql,sql,How many packages were received by accounts wi...,SELECT COUNT(DISTINCT cp.id) AS total_packages...
1,58456-ddl,ddl,None,CREATE TABLE delivery_facts (\n\tid INTEGER NO...
2,203101-sql,sql,What are the top 100 accounts with the highest...,"SELECT account_id_hashed, count(*) as total_pa..."
3,58455-ddl,ddl,None,CREATE TABLE delivery_preference (\n\taccount_...
4,239593-doc,documentation,None,de 'collo_packages' tabel bevat informatie ove...
5,58457-ddl,ddl,None,CREATE TABLE collo_packages (\n\tid INTEGER NO...


In [7]:
# some code to play around with
question = """
Wat is de verhouding tussen grote, middelgrote en kleine paketten?
"""
sql_code = test_querier.generate_sql(question)
print(sql_code)

sample_data = test_querier.generate_sample_data(sql_query=sql_code)
sample_data

WITH package_sizes AS (
    SELECT
        CASE
            WHEN ma_gewicht > 10 THEN 'groot'
            WHEN ma_gewicht > 5 THEN 'middelgroot'
            ELSE 'klein'
        END AS package_size
    FROM collo_packages
)
SELECT package_size, COUNT(*) AS total_packages
FROM package_sizes
GROUP BY package_size;


,package_size,total_packages
0,groot,794775
1,klein,205207
2,middelgroot,18
